In [161]:
import random
import nltk
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# Выбор модели

In [162]:
with open('BIG_BOT_CONFIG.json', 'r',encoding='utf-8') as f:
    BOT_CONFIG = json.load(f)

In [163]:
BOT_CONFIG

{'failure_phrases': ['Не могу ответить на ваш запрос, я еще учусь',
  'Ваш язык слишком сложный, попробуйте изменить вопрос',
  'Слишком сложно'],
 'intents': {'hello': {'examples': ['Хай',
    'Привет',
    'Добрый день',
    'Привет',
    'Хай',
    'Добрый день',
    'Доброго времени суток',
    'Добрый вечер',
    'Здравствуйте',
    'Даров',
    'Йоу',
    'Алоха',
    'Хеллоу',
    'Приветики',
    'Привет!',
    'Хай',
    'Добрый день',
    'Доброго времени суток',
    'Прив',
    'Хаюхай',
    'Здравствуйте',
    'Приветствую',
    'Здравия желаю',
    'Хелоу',
    'Приветики',
    'Привет',
    'Хай',
    'Хеллоу',
    'Привет!',
    'Хай!',
    'Добрый день',
    'Привет!',
    'Хай!',
    'Добрый день',
    'Привет!',
    'Здравствуй!',
    'Доброго времени суток!',
    'Привет!',
    'Хай!',
    'Добрый день',
    'ку',
    'Приветушки',
    'приветствую вас',
    'здарова',
    'рад видеть',
    'Привет!',
    'Хай',
    'Добрый день',
    'Привет!',
    'Хай!',
    'Добр

In [164]:
texts = []
intent_names = []

for intent, intent_data in BOT_CONFIG['intents'].items():
    for example in intent_data['examples']:
        texts.append(example)
        intent_names.append(intent)

In [165]:
# texts = []
# intent_names = []

# for intent, intent_data in BOT_CONFIG['intents'].items():
#     if 'inc_examples' in intent_data:
#         examples = list(set([example.lower() for example in intent_data['inc_examples']]))
#     else:
#         examples = list(set([example.lower() for example in intent_data['examples']]))
#     texts += examples
#     intent_names += [intent] * len(examples)

In [166]:
# results = []
# names = []
# models = []
# vectorizer = []

# models.append(('LR',LogisticRegression()))
# models.append(('SVC',SVC()))
# models.append(('LinearSVC',LinearSVC()))
# models.append(('RFC',RandomForestClassifier()))

# vectorizer.append(('CountVect',CountVectorizer()))
# vectorizer.append(('Tfidf',TfidfVectorizer()))

# for vname, vmodel in vectorizer:
#     vectorizer = vmodel
#     X = vectorizer.fit_transform(texts)
#     for name, model in models:
#         cv_results = cross_val_score(model,X,intent_names,cv=5,scoring='accuracy')
#         results.append(cv_results)
#         names.append(name)
#         msg = "%s %s: %f (%f)" %(vname, name, cv_results.mean(), cv_results.std())
#         print(msg)

In [167]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)
clf = LinearSVC();
clf.fit(X, intent_names)

LinearSVC()

In [168]:
def classify_intent(replica):
    intent = clf.predict(vectorizer.transform([replica]))[0]
    examples = BOT_CONFIG['intents'][intent]['examples']
    
    for example in examples:
        example = clear_text(example)
        if len(example) > 0:
            if abs(len(example)-len(replica))/len(example) < 0.5:
                distance = nltk.edit_distance(replica,example)
                if len(example) and distance/len(example) < 0.5:
                    return intent

In [169]:
def get_answer_by_intent(intent):
    if intent in BOT_CONFIG['intents']:
        responses = BOT_CONFIG['intents'][intent]['responses']
        return random.choice(responses)

#### Генеративная модель

In [170]:
with open('dialogues.txt',encoding='utf-8') as dialogues_file:
    dialogues_text = dialogues_file.read()
dialogues = dialogues_text.split('\n\n') 

In [171]:
def clear_text(text):
    text = text.lower()
    text = ''.join(char for char in text if char in 'абвгдеёжзийклмнопрстуфхцчшщъюьэюя -')
    return text

dataset = []
questions = set()

for dialogue in dialogues:
    replicas = dialogue.split('\n')
    replicas = replicas[:2]
    
    if len(replicas) == 2:
        question, answer = replicas
        question = clear_text(question[2:])
        answer = answer[2:]
        
        if len(question)>0 and question not in questions:
            questions.add(question)
            dataset.append([question,answer])     

In [172]:
dataset_by_word = {}

for question, answer in dataset:
    words = question.split(' ')
    for word in words:
        if word not in dataset_by_word:
            dataset_by_word[word] = []
        dataset_by_word[word].append([question,answer])

In [173]:
# # Самые употребляемые слова
# dataset_by_word_count = []

# for word, word_dataset in dataset_by_word.items():
#     dataset_by_word_count.append([word,len(word_dataset)])
    
# dataset_by_word_count.sort(key=lambda pair:pair[1], reverse=True)
# dataset_by_word_count[:100]

In [174]:
dataset_by_word_filtered = {}

for word, word_dataset in dataset_by_word.items():
    word_dataset.sort(key=lambda pair:len(pair[0]))    
    dataset_by_word_filtered[word] = word_dataset[:1000] # Берем по 1000 примеров вопросов для каждого слова

In [175]:
def generate_answer(replica):
    replica = clear_text(replica)    
    if not replica:
        return 
    
    words = set(replica.split(' '))
    words_dataset = []
    for word in words:
        if word in dataset_by_word_filtered:
            word_dataset = dataset_by_word_filtered[word]
            words_dataset += word_dataset
                
    results = [] #[[question,answer,distance],...]
    for question, answer in words_dataset:
        if abs(len(question)-len(replica))/len(question) < 0.2:            
            distance = nltk.edit_distance(replica,question)
            if distance / len(question) < 0.2:
                results.append([question,answer,distance])
    try:        
        question, answer, distance = min(results,key=lambda three: three[2])
        return answer
    except ValueError:
        return

In [176]:
def get_stub():
    failure_phrases = BOT_CONFIG['failure_phrases']
    return random.choice(failure_phrases)

In [177]:
stats = {'intents':0, 'generative':0, 'stubs':0}

In [178]:
def bot(replica):
    #NLU
    intent = classify_intent(replica)
    #Получение ответа
    if intent:
        answer = get_answer_by_intent(intent)
        if answer:
            stats['intents'] += 1
            return answer
    #Генеративная модель    
    answer = generate_answer(replica)
    if answer:
        stats['generative'] += 1
        return answer
    #Заглушка
    answer = get_stub()
    stats['stubs'] += 1
    return answer

In [179]:
print(bot('как житуха '))

Пойдет


In [180]:
stats

{'intents': 1, 'generative': 0, 'stubs': 0}

In [181]:
! pip install python-telegram-bot 

In [ ]:
from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, _: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, _: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def run_bot(update: Update, _: CallbackContext) -> None:
    """Echo the user message."""
    response = bot(update.message.text)
    update.message.reply_text(response)
    

def main()-> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("1774824416:AAFT8ej3QgkmSMQn1eHLz1LPSM2uuOdSkGQ", use_context=True)

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, run_bot))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()




In [ ]:
main()

In [ ]:
stats